# Simple oversampling - Pandas and imbalanced-learn
***

## Imports

In [1]:
import pandas as pd              # pandas for handling mixed data sets 

# imbalanced-learn for oversampling
from imblearn.over_sampling import RandomOverSampler                  

#### Create a sample data set

In [2]:
scratch_df = pd.DataFrame({'x1': pd.Series(np.arange(0, 10)),
                           'x2': pd.Series(np.random.randn(10)), 
                           'x3': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
                           'x4': ['C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'],
                           'y' : [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]})

scratch_df

NameError: name 'np' is not defined

#### Calculate Pearson correlation for numeric variables
`pandas.DataFrame.corr()` function shows that `x1` is much more correlated with `y` than `x2`.

In [ ]:
scratch_df.corr()

#### Calculate Chi-Square statistic for categorical variables
* `pandas.crosstab()` creates frequency tables
* `scipy.stats.chi2_contingency()` function on the contingency tables shows that the frequency of values in `x3` is related to `y` more so than the frequency of values in `x4`.

In [ ]:
pd.crosstab(scratch_df.x3, scratch_df.y)

In [ ]:
chi2, p, dof, ex = chi2_contingency(pd.crosstab(scratch_df.x3, scratch_df.y))
print('chi2 =', chi2)
print('p-value =', p)

In [ ]:
pd.crosstab(scratch_df.x4, scratch_df.y)

In [ ]:
chi2, p, dof, ex = chi2_contingency(pd.crosstab(scratch_df.x4, scratch_df.y))
print('chi2 =', chi2)
print('p-value =', p)

This simple variable selection exercise shows that `x1` and `x3` have observable relationships with `y`.

## Perform basic feature extraction

#### Create a sample data set

In [ ]:
scratch_df = pd.DataFrame({'x1': [1, 2.5, 3, 4.5],
                           'x2': [1.5, 2, 3.5, 4]})

scratch_df

#### Compress `x1` and `x2` into a single principal component

In [ ]:
pca = PCA(n_components=1)

In [ ]:
pca.fit(scratch_df)

Principal components analysis finds vectors that represent that direction(s) of most variance in a data set. These are called *eigenvectors*.

In [ ]:
print('First eigenvector = ', pca.components_)

Principal components are the projection of the data onto these eigenvectors. Principal components are usually centered around zero and each principal component is uncorrelated with all the others, i.e. principal components are *orthogonal* to one-another. Becuase prinicipal components represent the highest variance dimensions in the data and are not correlated with one another, they do an excellent job summarizing a data set with only a few dimensions (e.g. columns) and PCA is probably the most popular feature extraction technique.

In [ ]:
scratch_df['Centered_PC1'] = pca.transform(scratch_df[['x1', 'x2']])
scratch_df['Non_centered_PC1'] = pca.transform(scratch_df[['x1', 'x2']] + pca.mean_)
scratch_df['PC1_x1_back_projection'] = pd.Series(np.arange(1,8,2)) * pca.components_[0][0]
scratch_df['PC1_x2_back_projection'] = pd.Series(np.arange(1,8,2)) * pca.components_[0][1]
scratch_df

In [ ]:
x = plt.scatter(scratch_df.x1, scratch_df.x2, color='b')
pc, = plt.plot(scratch_df.PC1_x1_back_projection, scratch_df.PC1_x2_back_projection, color='r')
plt.legend([x, pc], ['Observed data (x)', 'First principal component projection'], loc=4)
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

## Proportional oversampling

#### Create a sample data set

In [ ]:
scratch_df = pd.DataFrame({'x': pd.Series(np.arange(0, 10)),
                           'y': [0, 1, 0, 0, 0, 0, 1, 0, 0, 0]})
scratch_df

If the event in a classification problem or the value in a prediction problem is imbalanced (usually toward zero) this can lead to biased models, single class predictions for classification models, and biased predictions for prediction models. The simplest approach for an imbalanced target is to *oversample* the data set.

In [ ]:
# fit random oversampling function
# cannot pass single array for X, must use numpy.reshape(-1, 1)
ros = RandomOverSampler()
over_sample_x, over_sample_y = ros.fit_sample(scratch_df.x.get_values().reshape(-1, 1), 
                                              scratch_df.y)

# create Pandas dataframe from oversampling results
over_sample_df = pd.DataFrame({'over_sample_x': over_sample_x.reshape(16,),
                               'over_sample_y': over_sample_y})
over_sample_df